In [1]:
import sys, os
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("Application"). \
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.hadoop.fs.s3a.access.key", f"{os.environ['AWS_ACCESS_KEY_ID']}").\
        config("spark.hadoop.fs.s3a.secret.key", f"{os.environ['AWS_SECRET_ACCESS_KEY']}").\
        config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", f"s3a://{os.environ['AWS_S3_BUCKET']}/spark-events/").\
        config("spark.history.fs.logDirectory", f"s3a://{os.environ['AWS_S3_BUCKET']}/spark-events/").\
        getOrCreate()


2023-12-17 09:53:34,635 INFO spark.SparkContext: Running Spark version 3.3.0
2023-12-17 09:53:34,769 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-12-17 09:53:34,864 INFO resource.ResourceUtils: ==============================================================
2023-12-17 09:53:34,864 INFO resource.ResourceUtils: No custom resources configured for spark.driver.
2023-12-17 09:53:34,864 INFO resource.ResourceUtils: ==============================================================
2023-12-17 09:53:34,865 INFO spark.SparkContext: Submitted application: Application
2023-12-17 09:53:34,893 INFO resource.ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 512, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
2023-12-17 09:53:3

In [7]:
conf = spark.sparkContext.getConf().getAll()

# Print Configuration
for k,v in conf:
    print(f"{k} : {v}")

spark.eventLog.enabled : true
spark.executor.memory : 512m
spark.driver.extraJavaOptions : -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED
spark.app.name : Application
spark.eventLog.dir : s3a://spark-events/
spark.app.submitTime : 1702803690431
spark.hadoop.fs.s3a.endpoint : http://172.22.0.2:9000
spark.executor.id : dr

In [2]:
df = spark.read.csv(f"s3a://{os.environ['AWS_S3_BUCKET']}/yellow_tripdata_2021-01.csv", header=True)

2023-12-17 09:54:43,424 INFO internal.SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
2023-12-17 09:54:43,427 INFO internal.SharedState: Warehouse path is 'file:/opt/workspace/code/spark-warehouse'.
2023-12-17 09:54:43,456 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@29fb52a3{/SQL,null,AVAILABLE,@Spark}
2023-12-17 09:54:43,457 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@4720d1e2{/SQL/json,null,AVAILABLE,@Spark}
2023-12-17 09:54:43,459 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@25ad430e{/SQL/execution,null,AVAILABLE,@Spark}
2023-12-17 09:54:43,460 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@612463b0{/SQL/execution/json,null,AVAILABLE,@Spark}
2023-12-17 09:54:43,463 INFO handler.ContextHandler: Started o.s.j.s.ServletContextHandler@3a4dd1a1{/static/sql,null,AVAILABLE,@Spark}
2023-12-17 09:54:47,050 INFO datasources.InMemoryFileIndex: It took 814 

In [3]:
df.show()

2023-12-17 09:55:39,284 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:55:39,284 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:55:39,284 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:55:39,311 INFO memory.MemoryStore: Block broadcast_3 stored as values in memory (estimated size 302.6 KiB, free 433.8 MiB)
2023-12-17 09:55:39,324 INFO memory.MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 53.5 KiB, free 433.7 MiB)
2023-12-17 09:55:39,325 INFO storage.BlockManagerInfo: Added broadcast_3_piece0 in memory on a31eab60218a:36769 (size: 53.5 KiB, free: 434.3 MiB)
2023-12-17 09:55:39,326 INFO spark.SparkContext: Created broadcast 3 from showString at NativeMethodAccessorImpl.java:0
2023-12-17 09:55:39,332 INFO execution.FileSourceScanExec: Pla

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

2023-12-17 09:55:45,386 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 6018 ms on 172.22.0.5 (executor 0) (1/1)
2023-12-17 09:55:45,386 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
2023-12-17 09:55:45,388 INFO scheduler.DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 6.038 s
2023-12-17 09:55:45,388 INFO scheduler.DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
2023-12-17 09:55:45,388 INFO scheduler.TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
2023-12-17 09:55:45,389 INFO scheduler.DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 6.045148 s
2023-12-17 09:55:45,478 INFO codegen.CodeGenerator: Code generated in 54.668617 ms


In [4]:
df.write.parquet(f"s3a://{os.environ['AWS_S3_BUCKET']}/yellow_tripdata/2021/01/", mode='overwrite')

2023-12-17 09:55:59,518 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:55:59,518 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:55:59,519 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:56:00,953 INFO parquet.ParquetFileFormat: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
2023-12-17 09:56:00,974 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 1
2023-12-17 09:56:00,975 INFO output.FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2023-12-17 09:56:00,976 INFO datasources.SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
2023-12-17 09:56:00,

In [5]:
df_parquet = spark.read.parquet(f"s3a://{os.environ['AWS_S3_BUCKET']}/yellow_tripdata/2021/01/")

2023-12-17 09:57:48,640 INFO datasources.InMemoryFileIndex: It took 380 ms to list leaf files for 1 paths.
2023-12-17 09:57:48,702 INFO spark.SparkContext: Starting job: parquet at NativeMethodAccessorImpl.java:0
2023-12-17 09:57:48,715 INFO scheduler.DAGScheduler: Got job 3 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
2023-12-17 09:57:48,716 INFO scheduler.DAGScheduler: Final stage: ResultStage 3 (parquet at NativeMethodAccessorImpl.java:0)
2023-12-17 09:57:48,716 INFO scheduler.DAGScheduler: Parents of final stage: List()
2023-12-17 09:57:48,716 INFO scheduler.DAGScheduler: Missing parents: List()
2023-12-17 09:57:48,718 INFO scheduler.DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[16] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
2023-12-17 09:57:48,720 INFO storage.BlockManagerInfo: Removed broadcast_4_piece0 on a31eab60218a:36769 in memory (size: 6.2 KiB, free: 434.1 MiB)
2023-12-17 09:57:48,729 INFO memory.MemorySt

In [14]:
df_parquet.show()

2023-12-17 09:06:35,820 INFO datasources.FileSourceStrategy: Pushed Filters: 
2023-12-17 09:06:35,821 INFO datasources.FileSourceStrategy: Post-Scan Filters: 
2023-12-17 09:06:35,821 INFO datasources.FileSourceStrategy: Output Data Schema: struct<VendorID: string, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: string, trip_distance: string ... 16 more fields>
2023-12-17 09:06:35,945 INFO codegen.CodeGenerator: Code generated in 65.567944 ms
2023-12-17 09:06:35,957 INFO memory.MemoryStore: Block broadcast_8 stored as values in memory (estimated size 306.1 KiB, free 433.1 MiB)
2023-12-17 09:06:35,972 INFO memory.MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 54.3 KiB, free 433.1 MiB)
2023-12-17 09:06:35,974 INFO storage.BlockManagerInfo: Added broadcast_8_piece0 in memory on a31eab60218a:43755 (size: 54.3 KiB, free: 434.1 MiB)
2023-12-17 09:06:35,975 INFO spark.SparkContext: Created broadcast 8 from showString at NativeMetho

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2021-01-01 00:30:10|  2021-01-01 00:36:12|              1|         2.10|         1|                 N|         142|          43|           2|          8|    3|    0.5|         0|           0|                  0.3

2023-12-17 09:06:36,835 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 5) in 796 ms on 172.22.0.6 (executor 1) (1/1)
2023-12-17 09:06:36,835 INFO scheduler.TaskSchedulerImpl: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2023-12-17 09:06:36,837 INFO scheduler.DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 0.829 s
2023-12-17 09:06:36,837 INFO scheduler.DAGScheduler: Job 4 is finished. Cancelling potential speculative or zombie tasks for this job
2023-12-17 09:06:36,837 INFO scheduler.TaskSchedulerImpl: Killing all running tasks in stage 4: Stage finished
2023-12-17 09:06:36,838 INFO scheduler.DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 0.834913 s


In [6]:
spark.stop()

2023-12-17 09:58:54,463 INFO server.AbstractConnector: Stopped Spark@1684c589{HTTP/1.1, (http/1.1)}{0.0.0.0:4041}
2023-12-17 09:58:54,469 INFO ui.SparkUI: Stopped Spark web UI at http://a31eab60218a:4041
2023-12-17 09:58:54,475 INFO cluster.StandaloneSchedulerBackend: Shutting down all executors
2023-12-17 09:58:54,475 INFO cluster.CoarseGrainedSchedulerBackend$DriverEndpoint: Asking each executor to shut down
2023-12-17 09:59:00,761 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
2023-12-17 09:59:00,781 INFO memory.MemoryStore: MemoryStore cleared
2023-12-17 09:59:00,782 INFO storage.BlockManager: BlockManager stopped
2023-12-17 09:59:00,786 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
2023-12-17 09:59:00,789 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
2023-12-17 09:59:00,805 INFO spark.SparkContext: Successfully stopped SparkContext
